In [1]:
import sys
sys.path.append('..\\..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
reload(gdspy)


<module 'gdspy' from 'C:\\ProgramData\\Anaconda3\\envs\\py38\\lib\\site-packages\\gdspy\\__init__.py'>

In [2]:
jc = 0.5e-6  # 1e-6 # uA/um^2
layers_configuration = {
    'total': 0,
    'restricted area': 10,
    'for removing': 100,
    'JJs': 1,
    'air bridges': 2,
    'air bridge pads': 3,
    'air bridge sm pads': 4,
    'vertical gridlines': 0,
    'horizontal gridlines': 0,
    'inverted': 101,
    'bandages': 20
}

sample = creator.Sample('simple_fluxonium', layers_configuration)

# specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size = 4.7e3
sample.chip_geometry.sample_horizontal_size = 4.7e3
chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads, 350)
sample.add(chip_edge_ground)

In [3]:
# qubit params
flux_w = 10
flux_s = 6
flux_g = 20

In [4]:
fluxonium_squid =  {'jjs_height': 0.1,
                             'jjs_width': 0.115,
                             'jj_lead_width':0.5,
                             'chain_width':1.0,
                             'ics': 0.1*0.115*jc,
                             'icb': 0.5*1.0*jc,
                             'chain_junctions': 45,
                             'lm': 3.3e-12,
                             'squid_orientation': 'horizontal',
                             'jjs_distance': 0.16,
                             'chain_jj_distance': 0.1,
                             'chain_top_offset': 1.0,
                             'flux_line_outer_extension': 11}

In [5]:
from QCreator.elements.gflux_squid import GFluxControllableSquid

sl_position = (2350, 2350)

fluxonium_w = 20
fluxonium_s = 15
fluxonium_g = 30

squid = GFluxControllableSquid(name='fluxonium-squid', 
                               position=sl_position, 
                               w=fluxonium_w, s=fluxonium_s, g=fluxonium_g,
                               layer_configuration=sample.layer_configuration, 
                               squid_params=fluxonium_squid,
                               flux_w=flux_w, flux_s=flux_s, flux_g=flux_g)

sample.add(squid)

{'jjs_height': 0.1, 'jjs_width': 0.115, 'jj_lead_width': 0.5, 'chain_width': 1.0, 'ics': 5.75e-09, 'icb': 2.5e-07, 'chain_junctions': 45, 'lm': 3.3e-12, 'squid_orientation': 'horizontal', 'jjs_distance': 0.16, 'chain_jj_distance': 0.1, 'chain_top_offset': 1.0, 'flux_line_outer_extension': 11}


In [6]:
fluxonium_cap_length = 400
open_ends = []
capacitors = []
for open_end_id, port_name in enumerate(['port1', 'port2']):
    position = -np.exp(1j*squid.get_terminals()[port_name].orientation)*fluxonium_cap_length
    position = np.asarray([np.real(position), np.imag(position)])
    position += squid.get_terminals()[port_name].position
    
    open_end = elements.OpenEnd(name='fluxonium-open-end'+str(open_end_id),
                                position=position,
                                 w=[squid.get_terminals()[port_name].w],
                                 s=[squid.get_terminals()[port_name].s, squid.get_terminals()[port_name].s],
                                 g=squid.get_terminals()[port_name].g,
                                 orientation=squid.get_terminals()[port_name].orientation+np.pi,
                                 layer_configuration=sample.layer_configuration,
                                 h1=20,
                                 h2=10)
    open_ends.append(open_end)
    sample.add(open_end)
    capacitor = sample.connect_cpw(open_end, squid, 'wide', port_name, points=[], 
                                   name='fluxonium-cap'+str(open_end_id))

In [7]:
sample.draw_design()
sample.watch()

In [8]:
tls, connections_flat, element_assignments = sample.get_tls()
omegas, γ, modes = tls.get_modes()
omegas/(2*np.pi)

array([2.63968085e+09, 4.62027710e+09, 1.50245721e+11, 1.50341745e+11,
       3.01095656e+11, 3.01143596e+11, 4.05504263e+11, 4.05517367e+11,
       4.68027634e+11, 4.68027634e+11, 4.68028275e+11, 4.68028275e+11,
       4.74583096e+11, 4.74613537e+11, 5.11696558e+11, 5.11696558e+11,
       5.11717516e+11, 5.11717516e+11])